# Tutorial on regexmodel

## Setup and installation

If you haven't installed `regexmodel` yet, including the optional dependencies, do so now:

In [1]:
# %pip install git+https://github.com/sodascience/regexmodel.git[tutorial]

Normally we would already have data that we want to model and synthesize, but for this tutorial we will use the faker package to generate that data for us. We will use fake email addresses.

In [92]:
from faker import Faker
from regexmodel.util import Dir
from regexmodel.datastructure import BaseRegex
from regexmodel.model2 import RegexModel
from regexmodel.model import RegexModel as OldRegexModel
from regexmodel.data2 import RegexNode, OrNode, Edge
import polars as pl

fake = Faker("nl")
Faker.seed(12345)
email_addresses = pl.Series([fake.ipv4() for _ in range(4000)])
series = email_addresses

In [93]:
model = RegexModel.fit(email_addresses, count_thres=20)
old_model = OldRegexModel.fit(email_addresses, count_thres=20)
print(model.fit_statistics(email_addresses))
#print(old_model.fit_statistics(email_addresses))

{'failed': 0, 'success': 4000, 'n_tot_char': 52985, 'n_char_success': 52985, 'n_parameters': 8, 'avg_log_like_per_char': -2.112847912712068, 'avg_log_like_pc_success': -2.112847912712068}


In [94]:
model.regex

'[0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}'

In [95]:
model.draw()

'225.5.517.3'

In [7]:
import numpy as np

def _get_bounds(key, count_dict, count_thres, sigma, tot_counts):
    if key in count_dict:
        lower_bound = (count_dict[key] - sigma*np.sqrt(count_thres))/tot_counts
        upper_bound = (count_dict[key] + sigma*np.sqrt(count_thres))/tot_counts
    else:
        lower_bound = 0
        upper_bound = (count_thres + sigma*np.sqrt(count_thres))/tot_counts
    return lower_bound, upper_bound
 

def _check_stat_compatible(stat_a, stat_b, count_thres, sigma=3):
    dict_stat_a = {regex._base_regex: len(series)-series.null_count() for regex, score, series in stat_a}
    dict_stat_b = {regex._base_regex: len(series)-series.null_count() for regex, score, series in stat_b}
    tot_a = np.sum([x for x in dict_stat_a.values()])
    tot_b = np.sum([x for x in dict_stat_b.values()])
    for key in (set(dict_stat_a) | set(dict_stat_b)):
        lower_a, upper_a = _get_bounds(key, dict_stat_a, count_thres, sigma, tot_a)
        lower_b, upper_b = _get_bounds(key, dict_stat_b, count_thres, sigma, tot_b)
        print(key, lower_a, upper_a, lower_b, upper_b)
        if lower_a > upper_b or lower_b > upper_a:
            return False
    return True

In [8]:
import string
isinstance(OrRegex([]), BaseRegex)

NameError: name 'OrRegex' is not defined

In [ ]:
from regexmodel.regexclass2 import UpperRegex, LowerRegex, DigitRegex, score_single, LiteralRegex, OrRegex
rclasses = [UpperRegex, LowerRegex, DigitRegex]
series = email_addresses

def get_class_stat(series, count_thres):
    score_list = []
    for rclass in rclasses:
        cur_class_stat = score_single(rclass._base_regex, series, count_thres, rclass.n_possible)
        if cur_class_stat[0] > 0:
            score_list.append((rclass(), *cur_class_stat))
    score_list.extend(LiteralRegex.get_candidates(series, count_thres))
    return sorted(score_list, key=lambda res: -res[1])


# cur_series = series
count_thres = 5
class_stat = get_class_stat(series, count_thres)
cur_best_regex, best_score, cur_best_series = class_stat[0]
cur_best_regex = OrRegex(cur_best_regex)

for cur_regex, score, cur_series in class_stat[1:]:
    if cur_best_regex.covers(cur_regex):
        continue
    cur_new_class_stat = get_class_stat(cur_series, count_thres)
    best_new_class_stat = get_class_stat(cur_best_series, count_thres)

    compatible = _check_stat_compatible(best_new_class_stat, cur_new_class_stat, count_thres)
    if compatible:
        if isinstance(cur_best_regex, OrRegex):
            cur_best_regex.append(cur_regex)
        else:
            cur_best_regex = OrRegex(cur_best_regex, cur_regex)

cur_series = series
start_non_null = len(series) - series.null_count()
cur_series = cur_best_regex.extract_after_first(series)
i_start = None
for cur_i in range(1, 100):
    compatible = cur_best_regex.check_compatibility(cur_series, count_thres)
    cur_non_null = len(cur_series) - cur_series.null_count()
    if not compatible or cur_non_null < count_thres or cur_non_null < count_thres:
        if i_start is None:
            i_start = cur_i-1
        i_end = cur_i-1
        break
    
    if cur_non_null < start_non_null + count_thres:
        i_start = cur_i
    cur_series = cur_best_regex.extract_after_first(cur_series)

print(i_start, i_end)
print(cur_best_regex.regex)
    # combi = OrRegex(cur_result[0], best_class_result[0])
    # combi_class_stat = score_single(combi.regex, series, count_thres, combi.n_possible)
    # combi_new_class_stat = get_class_stat(cur_result[2], count_thres)  
    # print(f"-----{cur_result[0].regex}---{compatible}---{best_class_result[0].covers(cur_result[0])}----")
# best_class_result

In [ ]:
import json
with open("../scripts/data/benchmark.json") as handle:
    bench_data = json.load(handle)
with open("../scripts/data2/benchmark.json") as handle:
    bench_data_new = json.load(handle)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def _get_success_rate(fake_type, bench_data):
    n_param = []
    success = []
    fails = []
    # for var_data in bench_data.values():
    for data in bench_data[fake_type].values():
        n_param.extend(x["n_parameters"] for x in data)
        success.extend(x["success"] for x in data)
        fails.extend(x["failed"] for x in data)

    success_rate = np.array(success)/(np.array(fails)+np.array(success))
    return np.array(n_param), success_rate

def plot_succes_rate(fake_type):
    plt.scatter(*_get_success_rate(fake_type, bench_data), label="old")
    plt.scatter(*_get_success_rate(fake_type, bench_data_new), label="new")
    plt.legend()
    plt.show()

In [ ]:
plot_succes_rate("job")

In [ ]:
for url in email_addresses[:10]:
    print(url, list(model.regex_edge.log_likelihood(url)))

In [ ]:

def create_regex_from_list(regex_data):
    if isinstance(regex_data, BaseRegex):
        return regex_data.regex
    if isinstance(regex_data, tuple):
        optional_regex = [create_regex_from_list(rx) for rx in regex_data]
        return r"(" + r"|".join(optional_regex) + ")"
    return "".join(create_regex_from_list(rx) for rx in regex_data)



## Modeling the structured strings

Now we will use the regexmodel package to model the data:

In [ ]:
    # while cur_series.drop_nulls().len() > count_thres:
        
    # if result["score"] < count_thres/len(series):
    #     return [], []
    # regex_list, count_list = fit_main_branch(result["new_series"], count_thres=count_thres,
    #                                          direction=direction)
    # n_main_line = result["new_series"].drop_nulls().len()
    # n_alt_line = result["alt_series"].drop_nulls().len()
    # regex_list = [result["regex"]] + regex_list
    # count_list = [n_main_line] + count_list

    # if len(result["alt_series"].drop_nulls()) > count_thres and optionals:

    #     regex_main = "".join(rx.regex for rx in regex_list)
    #     res = result["alt_series"].str.extract(r"(^[\S\s]*?)" + regex_main + r"$")
    #     alt_regex_list, alt_count_list = fit_main_branch(res, count_thres, direction)
    #     if len(alt_regex_list) > 0:
    #         regex_list = [(alt_regex_list, [])] + regex_list
    #         count_list = [(alt_count_list[-1], n_main_line)] + count_list

    # return return_links
    # # return regex_list, count_list


In [ ]:
from regexmodel import RegexModel

model = RegexModel.fit(email_addresses)

Let's first see how the good the model is by synthesizing new email addresses:

In [ ]:
[model.draw() for _ in range(10)]

While certainly not perfect, it certainly isn't so bad either, given that we have given the model only positive examples!

Now let's look at the serialization of the model:

In [ ]:
model.serialize()

The serialization might seem overwhelming at first, but the first regex (`[a-z]{3,18}[0-9]{2,2}[@][a-z]{4,9}[\\\\.][c][o][m]`) is usually the most important one. We call this the main branch. On this main branch, there will be side branches, for example for ".info" and ".biz" email addresses.

## Modeling performance

There are also some modeling statistics that can be computed. Note that computing these can take a while depending on your computer.

In [ ]:
model.fit_statistics(email_addresses)

What the `fit_statistics` method does is to retrace back whether an email address that is given to it (e.g. johndoe@example.com) has a non-zero probability to be generated by the regex model. As we can see above, there were 18 email addresses in the list that have a probability of 0 to be generated by the model, while the overwhelming majority (982) can be generated with the fitted model.

The value `n_parameters` gives the number of nodes in the model, and is thus an indicator of the complexity of the model. This is also correlated with the fit taking longer. We can influence this parameter during fitting by setting the `count_thres` parameter. If we set that threshold higher, we generally have a lower number of parameters and better performance.

The statistic `avg_log_like_per_char` (average log-likelihood per character) shows how probable a value is on average per character. To understand this better, let's take a more simple example, where the regex is simply `\d{2,2}`. For this regex, the log likelihood is simply log(1/10\*1/10) = -2\*log(10). Since all values have 2 characters, the average log-likelihood per character is -log(10) ~= 2.30. For failed values (values that cannot be generated by the model), we use a penalty score of -log(1000) per character.

Ideally we want to have the lowest `n_parameters` (simplest model) with the highest `success` and the highest log-likelihood.

## Visualization

To more clearly understand how the graph looks like, we can plot the regex model using the `regex_model_to_pyvis` function. To retrace the paths that can be taken, first find the start node and find the main branch.

Note: PyVis doesnt work interactively in VSCode/Code OSS.

In [ ]:
from regexmodel.visualization import regex_model_to_pyvis

net = regex_model_to_pyvis(model)
net.show("regex.html", notebook=True)